In [1]:
import gymnasium as gym
from pogema import GridConfig
from stable_baselines3 import DQN, A2C
from stable_baselines3.common.evaluation import evaluate_policy

%load_ext autoreload
%autoreload 2
%matplotlib inline

grid_config = GridConfig(
    size=8,
    density=0.3,
    num_agents=1,
    max_episode_steps=64
)

env = gym.make("Pogema-v0",grid_config=grid_config)

a2c_model = A2C(
    "MlpPolicy",
    env,
    verbose=1
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -1.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


#### Optuna Integration

In [2]:
""" Optuna example that optimizes the hyperparameters of
a reinforcement learning agent using A2C implementation from Stable-Baselines3
on a Gymnasium environment.

This is a simplified version of what can be found in https://github.com/DLR-RM/rl-baselines3-zoo.

You can run this example as follows:
    $ python sb3_simple.py

"""
from typing import Any
from typing import Dict

import gymnasium
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from stable_baselines3 import A2C
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
import torch
import torch.nn as nn


N_TRIALS = 100
N_STARTUP_TRIALS = 5
N_EVALUATIONS = 2
N_TIMESTEPS = int(1.0e4)
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_EPISODES = 3


DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    "verbose": 1,
    "env": env
}

def sample_a2c_params(trial: optuna.Trial) -> Dict[str, Any]:
    """Sampler for A2C hyperparameters."""
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)

    trial.set_user_attr("gamma", gamma)

    return {
        "gamma": gamma,
        "learning_rate": learning_rate,
    }


class TrialEvalCallback(EvalCallback):
    """Callback used for evaluating and reporting a trial."""

    def __init__(
        self,
        eval_env: gymnasium.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):
        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            super()._on_step()
            self.eval_idx += 1
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need.
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True


def objective(trial: optuna.Trial) -> float:
    kwargs = DEFAULT_HYPERPARAMS.copy()
    # Sample hyperparameters.
    kwargs.update(sample_a2c_params(trial))
    # Create the RL model.
    model = A2C(**kwargs)
    # Create env used for evaluation.
    eval_env = Monitor(env)
    # Create the callback that will periodically evaluate and report the performance.
    eval_callback = TrialEvalCallback(
        eval_env, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ, deterministic=True
    )

    nan_encountered = False
    try:
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN.
        print(e)
        nan_encountered = True
    finally:
        # Free memory.
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed.
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward


if __name__ == "__main__":
    # Set pytorch num threads to 1 for faster training.
    torch.set_num_threads(1)

    sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
    # Do not prune before 1/3 of the max budget is used.
    pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3)

    study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
    try:
        study.optimize(objective, n_trials=N_TRIALS, n_jobs=4, timeout=600)
    except KeyboardInterrupt:
        pass

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print("    {}: {}".format(key, value))


/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-11-10 17:04:43,433] A new study created in memory with name: no-name-429a8ea4-4bf9-4968-95c0-a65f0b107470
[W ParallelNative.cpp:230] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 53.6      |
|    ep_rew_mean        | 0.444     |
| time/                 |           |
|    fps                | 234       |
|    iterations         | 100       |
|    time_elapsed       | 2         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.61     |
|    explained_variance | -1.17e+05 |
|    learning_rate      | 0.000114  |
|    n_updates          | 99        |
|    policy_loss        | 0.285     |
|    value_loss         | 0.0753    |
----------------

[I 2023-11-10 17:05:27,437] Trial 2 finished with value: 0.0 and parameters: {'gamma': 0.0003476973944621764, 'lr': 0.0376067695210782}. Best is trial 2 with value: 0.0.


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| eval/                 |          |
|    mean_ep_length     | 37.7     |
|    mean_reward        | 0.333    |
| time/                 |          |
|    total_timesteps    | 10000    |
| train/                |          |
|    entropy_loss       | -1.61    |
|    explained_variance | 0        |
|    learning_rate      | 0.000114 |
|    n_updates          | 1999     |
|    policy_loss        | -0.165   |
|    value_loss         | 0.0109   |
------------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 48.8     |
|    ep_rew_mean     | 0.48     |
| time/              |          |
|    fps             | 221      |
|    iterations      | 2000     |
|    time_elapsed    | 45       |
|    total_timesteps | 10000    |
---------------------------------
-------------------------------------
| eval/   

[I 2023-11-10 17:05:28,627] Trial 3 finished with value: 0.0 and parameters: {'gamma': 0.0003802282720031166, 'lr': 0.00012098464019917173}. Best is trial 2 with value: 0.0.
[I 2023-11-10 17:05:28,640] Trial 1 finished with value: 0.3333333333333333 and parameters: {'gamma': 0.0020336678930584252, 'lr': 0.00011441085777384108}. Best is trial 1 with value: 0.3333333333333333.


...

------------------------------------
| eval/                 |          |
|    mean_ep_length     | 55.3     |
|    mean_reward        | 0.333    |
| time/                 |          |
|    total_timesteps    | 10000    |
| train/                |          |
|    entropy_loss       | -1.61    |
|    explained_variance | -39.5    |
|    learning_rate      | 1e-05    |
|    n_updates          | 1999     |
|    policy_loss        | -0.698   |
|    value_loss         | 0.234    |
------------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50.4     |
|    ep_rew_mean     | 0.28     |
| time/              |          |
|    fps             | 238      |
|    iterations      | 2000     |
|    time_elapsed    | 41       |
|    total_timesteps | 10000    |
---------------------------------


[I 2023-11-10 17:14:55,632] Trial 54 finished with value: 0.3333333333333333 and parameters: {'gamma': 0.06127288433187494, 'lr': 1.0045117614749572e-05}. Best is trial 24 with value: 1.0.


-------------------------------------
| eval/                 |           |
|    mean_ep_length     | 64        |
|    mean_reward        | 0         |
| time/                 |           |
|    total_timesteps    | 10000     |
| train/                |           |
|    entropy_loss       | -1.61     |
|    explained_variance | -2.31e+04 |
|    learning_rate      | 2.44e-05  |
|    n_updates          | 1999      |
|    policy_loss        | 0.163     |
|    value_loss         | 0.026     |
-------------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 44.2     |
|    ep_rew_mean     | 0.38     |
| time/              |          |
|    fps             | 246      |
|    iterations      | 2000     |
|    time_elapsed    | 40       |
|    total_timesteps | 10000    |
---------------------------------


[I 2023-11-10 17:14:55,982] Trial 55 finished with value: 0.0 and parameters: {'gamma': 0.05309843977384957, 'lr': 2.441799698747516e-05}. Best is trial 24 with value: 1.0.


Number of finished trials:  56
Best trial:
  Value:  1.0
  Params: 
    gamma: 0.042613270090891706
    lr: 3.5229093454971854e-05
  User attrs:
    gamma: 0.9573867299091083


### Train agent with best hyper parameters

In [3]:
a2c_model = A2C(
    "MlpPolicy",
    env,
    verbose=1,
    gamma=1-0.042613270090891706,
    learning_rate=3.5229093454971854e-05,
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [4]:
# Train agent and save it
a2c_model.learn(int(1.0e4))
a2c_model.save("saved/a2c_a_mini")

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 50.3      |
|    ep_rew_mean        | 0.222     |
| time/                 |           |
|    fps                | 707       |
|    iterations         | 100       |
|    time_elapsed       | 0         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.61     |
|    explained_variance | -1.35e+03 |
|    learning_rate      | 3.52e-05  |
|    n_updates          | 99        |
|    policy_loss        | 0.287     |
|    value_loss         | 0.145     |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 49.8     |
|    ep_rew_mean        | 0.3      |
| time/                 |          |
|    fps                | 672      |
|    iterations         | 200      |
|    time_elapsed       | 1        |
|    total_timesteps    | 1000     |
| train/             

#### Testing 

In [5]:
a2c_model = A2C.load("saved/a2c_a_mini")

env.reset()

mean_reward, std_reward = evaluate_policy(a2c_model, env, deterministic=True, n_eval_episodes=20)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:0.05 +/- 0.22


In [6]:
# RANDOM SEED
from IPython.display import SVG, display
from pogema.animation import AnimationMonitor, AnimationConfig

env = AnimationMonitor(env)

def evaluate_success_rate(model, env, num_episodes=1000):
    success_count = 0
    step_array = []
    for i in range(num_episodes):
        print(f'---{i}---')
        obs = env.reset()

        # Check if observation is a tuple and extract the first element if true.
        if isinstance(obs, tuple):
            obs = obs[0]
        max_step = 64
        steps_taken = 0
        done = truncated = False
        while not done and max_step > 0:
            action, _ = model.predict(obs)
            next_obs, reward, done, truncated, info = env.step(action)
            print(action,max_step,success_count,done)
            max_step -= 1
            steps_taken += 1
            # Check if next_obs is a tuple and extract the first element if true.
            if isinstance(next_obs, tuple):
                next_obs = next_obs[0]
            obs = next_obs

            # Check if agent was successful in that episode.
            if done:
                success_count += 1
                step_array.append(steps_taken)
                env.save_animation(f"render{i}.svg", AnimationConfig(egocentric_idx=0))
                break

    success_rate = success_count / num_episodes
    return success_rate, step_array

success_rate,step_array = evaluate_success_rate(a2c_model, env)
print(f"Agent Success Rate: {success_rate * 100:.2f}%")
print(f"steps to termination : {step_array}")

---0---
2 64 0 False
1 63 0 False
2 62 0 False
2 61 0 False
0 60 0 False
3 59 0 True
---1---
4 64 1 False
2 63 1 False
3 62 1 False
3 61 1 False
1 60 1 False
1 59 1 False
0 58 1 False
2 57 1 False
3 56 1 False
3 55 1 False
4 54 1 False
3 53 1 False
4 52 1 False
2 51 1 False
4 50 1 False
1 49 1 False
0 48 1 False
1 47 1 False
3 46 1 False
3 45 1 False
0 44 1 False
0 43 1 False
4 42 1 False
2 41 1 False
0 40 1 False
2 39 1 False
1 38 1 False
2 37 1 False
1 36 1 False
2 35 1 False
0 34 1 False
4 33 1 False
3 32 1 False
3 31 1 False
1 30 1 False
4 29 1 False
3 28 1 False
2 27 1 False
4 26 1 False
2 25 1 False
2 24 1 False
1 23 1 False
2 22 1 False
4 21 1 False
2 20 1 False
0 19 1 False
4 18 1 False
4 17 1 False
0 16 1 False
3 15 1 False
2 14 1 False
1 13 1 False
0 12 1 False
0 11 1 False
2 10 1 False
1 9 1 False
3 8 1 False
0 7 1 False
4 6 1 False
1 5 1 False
1 4 1 False
0 3 1 False
1 2 1 False
1 1 1 False
---2---
1 64 1 False
4 63 1 True
---3---
2 64 2 False
1 63 2 False
3 62 2 False
4 61